In [1]:
from gensim.models.keyedvectors import KeyedVectors


model = KeyedVectors.load_word2vec_format('covid-19.embeddings')

In [2]:
import pickle
with open("tokenizer.pickle", "rb") as f:
    tokenizer = pickle.load(f)
    
with open('document_mappping.pkl', "rb") as f:
    documents = pickle.load(f)
    
    
word_idx = tokenizer.word_index
inv_map = {v: k for k, v in word_idx.items()}

In [3]:
def similarity_finder(inp_id):
    results = []
    similarity = model.most_similar(str(inp_id), topn=1000)
    for similar in similarity:
        if not similar[0].startswith("d"):
            word = inv_map[int(similar[0])]
            results.append((word, similar[1]))
        else:
            results.append((documents[similar[0]], similar[1]))
    return results

In [4]:
similarity_finder("d100")

[('The virus specific intracellular RNA specie two murine coronaviruses MHV A59 MHV JHM Seven virus specific polyadenylated RNA specie identify mouse cell infect murine coronaviruses MHV A59 A59V MHV JHM JHMV MHV infect 17CL·1 cell label 32P orthophosphate presence actinomycin cytoplasmic RNA extract analyze agarose gel electrophoresis These RNA specie range size dalton The A59V JHMV specific RNAs identical molecular weight comigrate agarose gel The large intracellular RNA specie identical RNA isolate purified virion determine agarose gel electrophoresis oligonucleotide fingerprint study ribonuclease digest Oligonucleotide fingerprint six subgenomic RNAS show sequence contain present virion RNA confirm virus specific nature The fingerprinting study also demonstrate six subgenomic RNA specie make nested set The sequence present RNA specie also present large RNA specie These large RNAs also contain additional sequence consistent great size The subgenomic RNAs fulfull many criterion mRNAs

## Word closeness

In [5]:
all_words =tokenizer.word_counts

In [6]:
all_words_freq = sorted(list(all_words.items()), key=lambda x: x[1])[-2000:]

In [10]:
all_words_freq = [x[0] for x in all_words_freq]

In [8]:
import gensim
import numpy as np
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
import plotly.offline as plt
import plotly.graph_objs as go
plt.init_notebook_mode(connected=True)

In [8]:
from sklearn.cluster import KMeans
import numpy as np


def get_coordinates(model, words):
    X = []
    arr = np.empty((0,64), dtype='f')
    labels = []
    for wrd_score in words:
        try:
            X.append(model[str(word_idx[wrd_score])])
            arr = np.append(arr, np.array([model[str(word_idx[wrd_score])]]), axis=0)
            labels.append(wrd_score)
        except Exception as e:
            print(e)
            pass
  
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    return x_coords, y_coords, labels, X

In [11]:
x, y, labels, X = get_coordinates(model, all_words_freq)

In [12]:
kmeans = KMeans(n_clusters=20, random_state=0).fit(X)

In [12]:
plot = [go.Scatter(x = x,
                    y = y,
                    mode = 'markers+text',
                    text = labels,
                    textposition='bottom center',
                    hoverinfo = 'text',
                    marker=dict(size=5,opacity=0.8, color=kmeans.labels_))]

layout = go.Layout(title='prod cat projection')
# fig = go.Figure(data=plot, layout=layout)
# plt.iplot(fig)

NameError: name 'kmeans' is not defined

## Document similarity

In [9]:
from sklearn.cluster import KMeans
import numpy as np


def get_coordinates(model, words):
    X = []
    arr = np.empty((0,64), dtype='f')
    labels = []
    for wrd_score in words:
        try:
            X.append(model[wrd_score])
            arr = np.append(arr, np.array([model[wrd_score]]), axis=0)
            labels.append(wrd_score)
        except Exception as e:
            print(e)
            pass
  
    tsne = TSNE(n_components=3, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    z_coords = Y[:, 2]
    return x_coords, y_coords,z_coords, labels, X

In [6]:
words = ["d"+str(i) for i in range(18500)]

In [10]:
x, y,z_coords, labels, X = get_coordinates(model, words)

In [11]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)

In [12]:
km_labels = list(kmeans.labels_)

In [13]:
# indices = [i for i, x in enumerate(kmeans.labels_) if x == 2]
len(km_labels)

18500

In [14]:
import random
X_new, y_new,z_new, labels_new = [], [], [],[]

kmeans_labels = []

for i in range(10):
    indices =[]
    for idx in range(len(km_labels)):
        if km_labels[idx] == i:
            indices.append(idx)
    print(len(indices))
    if len(indices) > 20:
        idx = random.sample(indices, 20)
        for index in idx:
            X_new.append(x[index])
            y_new.append(y[index])
            z_new.append(z_coords[index])
            labels_new.append(labels[index])
            kmeans_labels.append(i)
    else:
        for index in indices:
            X_new.append(x[index])
            y_new.append(y[index])
            z_new.append(z_coords[index])
            labels_new.append(labels[index])
            kmeans_labels.append(i)

2464
1669
2493
1455
1349
1728
2304
2481
1332
1225


In [15]:
hover = [documents[i] for i in labels_new]

In [16]:
def wrap_by_word(s, n):
    '''returns a string where \\n is inserted between every n words'''
    a = s.split()
    ret = ''
    for i in range(0, len(a), n):
        ret += ' '.join(a[i:i+n]) + '\n'

    return ret


hover = [wrap_by_word(x, 8) for x in hover]

In [17]:
hover = [x.replace("\n", "<br>") for x in hover]

In [18]:
plot = [go.Scatter3d(x = X_new,
                    y = y_new,
                    z=z_new,
                    mode = 'markers+text',
                    text = labels_new,
                    hovertext=hover,
                    textposition='bottom center',
                    marker=dict(size=5,opacity=0.8, color=kmeans_labels))]

layout = go.Layout(title='prod cat projection')


In [19]:
fig = go.Figure(data=plot, layout=layout)
plt.iplot(fig)